In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from definitions import REPO_ROOT
import src.data.preprocess_data as prep
from src.data.data_loader import RepeatedStratifiedKFoldDataloader
import src.data.var_names as abcd_vars

RANDOM_STATE = 77

In [2]:
abcd_data = pd.read_csv(REPO_ROOT / 'data' / 'processed' / 'abcd_data.csv', index_col='src_subject_id')
print(abcd_data.shape)
abcd_data.head()

(8592, 304)


,age,female,married,race_ethnicity_Asian,race_ethnicity_Black,race_ethnicity_Hispanic,race_ethnicity_Other,race_ethnicity_White,high.educ_< HS Diploma,high.educ_Bachelor,...,Major Depressive Disorder,Bipolar Disorder,Psychotic Symptoms,ADHD,Oppositional Defiant Disorder,Conduct Disorder,PTSD,Obsessive Compulsive Disorder,Sleep Problems,Suicide Tendency
src_subject_id,,,,,,,,,,,,,,,,,,,,,
NDAR_INV007W6H7B,126,0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
NDAR_INV00CY2MDM,130,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
NDAR_INV00HEV6HB,124,0,1,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
NDAR_INV00LJVZK2,121,0,0,0,0,0,1,0,1,0,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
NDAR_INV00NPMHND,118,1,1,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [3]:
abcd_data = prep.select_one_child_per_family(
    abcd_data_path=REPO_ROOT / 'data' / 'raw',
    abcd_df=abcd_data,
    random_state=RANDOM_STATE
)
abcd_data.shape

(7191, 304)

In [66]:
data_loader = RepeatedStratifiedKFoldDataloader(
    dataframe=abcd_data,
    features=abcd_vars.all_brain_features.features,
    responses=abcd_vars.diagnoses.features,
    confounders=abcd_vars.sociodem.features,
    n=2,
    k=3,
    val_ratio=0.2,
    random_state=RANDOM_STATE
)

In [5]:
from src.models.classifier_chain import ValidationClassifierChain, LogisticRegressionModel
from src.models.evaluate import MultilabelBinaryEvaluator
from src.models.evaluate import BinaryEvaluator
from sklearn.linear_model import LogisticRegression

rocauc_cc = {d: [] for d in abcd_vars.diagnoses.features}
rocauc_c = {d: [] for d in abcd_vars.diagnoses.features}

for train, valid, test, features_selected in data_loader:

    # Classifier chain
    cc = ValidationClassifierChain(
        model=LogisticRegressionModel,
        features=features_selected,
        responses=abcd_vars.diagnoses.features,
        random=True,
        random_state=RANDOM_STATE
    )
    cc.fit(train, valid)
    preds = cc.predict_proba(test)

    # Single classifier
    ml_evaluator = MultilabelBinaryEvaluator(test[abcd_vars.diagnoses.features], preds)
    for key, item in ml_evaluator.roc_auc().items():
        rocauc_cc[key].append(item)
    
    for diagnosis in abcd_vars.diagnoses.features:
        lr_model = LogisticRegression(
            solver='lbfgs',
            max_iter=300,
            class_weight='balanced',
            random_state=RANDOM_STATE
        )
        total = pd.concat((train, valid))
        lr_model.fit(total[features_selected], total[diagnosis])
        y_pred = lr_model.predict_proba(test[features_selected])[:, 1]
        
        b_evaluator = BinaryEvaluator(test[diagnosis], y_pred)
        rocauc_c[diagnosis].append(b_evaluator.roc_auc)

C:\ProgramData\Anaconda3\envs\abcd\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\envs\abcd\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\envs\abcd\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\envs\abcd\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\P

In [55]:
for diagnosis in abcd_vars.diagnoses.features:
    print(diagnosis)
    print(f"   CC: {np.mean(rocauc_cc[diagnosis])}, C: {np.mean(rocauc_c[diagnosis])}")

Major Depressive Disorder
   CC: 0.510393437925121, C: 0.5056447508413373
Bipolar Disorder
   CC: 0.5382346352238345, C: 0.5455665636407838
Psychotic Symptoms
   CC: 0.47665136415710074, C: 0.47665136415710074
ADHD
   CC: 0.5258638170370881, C: 0.5324069215489032
Oppositional Defiant Disorder
   CC: 0.5082938246333186, C: 0.5110733840370186
Conduct Disorder
   CC: 0.506704084482577, C: 0.5051865322740404
PTSD
   CC: 0.46955046574160647, C: 0.47618565908419874
Obsessive Compulsive Disorder
   CC: 0.48853727286961246, C: 0.5057472879231789


In [11]:
from sklearn.multioutput import ClassifierChain

rocauc_skcc = {d: [] for d in abcd_vars.diagnoses.features}

for train, valid, test, features_selected in data_loader:
    base_lr = LogisticRegression(
        solver='lbfgs',
        max_iter=300,
        class_weight='balanced',
        random_state=RANDOM_STATE
    )
    chain = ClassifierChain(base_lr, order='random', random_state=RANDOM_STATE)
    total = pd.concat((train, valid))
    chain.fit(
        total[features_selected],
        total[abcd_vars.diagnoses.features]
    )
    y_pred = chain.predict_proba(test[features_selected])
    
    for i, d in enumerate(abcd_vars.diagnoses.features):
        evaluator = BinaryEvaluator(test[d], y_pred[:, i])
        rocauc_skcc[d].append(evaluator.roc_auc)

C:\ProgramData\Anaconda3\envs\abcd\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\envs\abcd\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\envs\abcd\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\envs\abcd\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\P

In [13]:
for diagnosis in abcd_vars.diagnoses.features:
    print(diagnosis)
    print(f"   Sklearn CC: {np.mean(rocauc_skcc[diagnosis])}")

Major Depressive Disorder
   Sklearn CC: 0.510393437925121
Bipolar Disorder
   Sklearn CC: 0.5382346352238345
Psychotic Symptoms
   Sklearn CC: 0.47665136415710074
ADHD
   Sklearn CC: 0.5258638170370881
Oppositional Defiant Disorder
   Sklearn CC: 0.5082938246333186
Conduct Disorder
   Sklearn CC: 0.506704084482577
PTSD
   Sklearn CC: 0.46955046574160647
Obsessive Compulsive Disorder
   Sklearn CC: 0.48853727286961246


### Saving data and displaying results

In [95]:
from pathlib import Path
import json
from pytorch_lightning.loggers import TensorBoardLogger

In [114]:
class ResultManager:
    
    def __init__(self,
                 tensorboard_logger,
                 save_root: Path,
                 save_params: dict = None):
        """Utility object that saves raw predictions, calculates and saves ROC
        AUC values, and writes to TensorBoard.
        
        :param save_root: Root directory of current experiment run
        :param roc_auc_values: Dictionary holding ROC AUC values of each fold
        :param logger: TensorBoard logger to write to
        """
        i = 0
        while (save_root / f"run_{i}").exists():
            i += 1
        self.save_root = save_root / f"run_{i}"
        self.save_root.mkdir(parents=True)
        # Save high level parameters (e.g. random_state)
        if save_params:
            with open(self.save_root / 'params.txt', 'w') as file:
                file.write(json.dumps(save_params))
        # Store ROC AUC values
        self.roc_auc_values = {}
        self.logger = tensorboard_logger
        
    def save_predictions(self,
                         dataset_name: str,
                         model_name: str,
                         fold: int,
                         split_set: str,
                         y_true: pd.DataFrame,
                         y_pred: pd.DataFrame) -> None:
        """Does two things:
         1. Saves raw predictions y_pred
         2. Computes ROC AUC values and stores them internally to save them
            later when finish() is called
           
        :param dataset_name: Name of full dataset (e.g. unpermuted, permuted_1,
            etc.)
        :param model_name: Name of model
        :param fold: Number of current fold in (repeated) k-fold CV
        :param split_set: Name of split set. Possible values: train, valid,
            test
        :param y_true: True labels of dataset
        :param y_pred: Predicted labels of dataset
        :return:
        """
        path = self.save_root / dataset_name / model_name / split_set
        if not path.exists():
            path.mkdir(parents=True)
        y_pred.to_csv(path / f"fold_{fold}.csv", index=True)
        # Compute and store ROC AUC value
        #   Set up ROC AUC dictionary
        if dataset_name not in self.roc_auc_values.keys():
            self.roc_auc_values[dataset_name] = {}
        if model_name not in self.roc_auc_values[dataset_name].keys():
            self.roc_auc_values[dataset_name][model_name] = {}
        if split_set not in self.roc_auc_values[dataset_name][model_name].keys():
            self.roc_auc_values[dataset_name][model_name][split_set] = {}
        #   Compute ROC AUC values
        mlbe = MultilabelBinaryEvaluator(y_true=y_true, y_pred=y_pred)
        self.roc_auc_values[dataset_name][model_name][split_set][fold] = mlbe.roc_auc()
        #   Write to TensorBoard
        for key in self.roc_auc_values[dataset_name][model_name][split_set][fold].keys():
            # Track ROC AUC values
            self.logger.experiment.add_scalar(
                f"{dataset_name}/{model_name}/{split_set}/ROC_AUC_mean_{key}",
                self.roc_auc_values[dataset_name][model_name][split_set][fold][key],
                global_step=fold
            )
            # Track histogram of ROC AUC values
            roc_auc_list = []
            for idx in self.roc_auc_values[dataset_name][model_name][split_set].keys():
                roc_auc_list.append(
                    self.roc_auc_values[dataset_name][model_name][split_set][idx][key]
                )
            self.logger.experiment.add_histogram(
                f"{dataset_name}/{model_name}/{split_set}/ROC_AUC_{key}",
                np.array(roc_auc_list),
                global_step=fold
            )
        
    def finish(self) -> None:
        """
        Saves ROC AUC values as dataframes. Call this function after training
        is finished.
        """
        for dataset_name, item0 in self.roc_auc_values.items():
            for model_name, item1 in item0.items():
                for split_set, item2 in item1.items():
                    path = self.save_root / dataset_name / model_name / split_set
                    roc_auc_df = pd.DataFrame.from_dict(item2, orient='index')
                    roc_auc_df.to_csv(path / 'roc_auc.csv', index=True)

In [115]:
tensorboard_logger = TensorBoardLogger(REPO_ROOT / 'tensorboard')

In [116]:
manager = ResultManager(
    tensorboard_logger=tensorboard_logger,
    save_root=REPO_ROOT / 'results',
    save_params={
        'random_state': RANDOM_STATE, 'n': data_loader.n, 'k': data_loader.k
    }
)

auc_dict = {d: [] for d in abcd_vars.diagnoses.features}

for i, (train, valid, test, features_selected) in enumerate(data_loader):

    # Classifier chain
    cc = ValidationClassifierChain(
        model=LogisticRegressionModel,
        features=features_selected,
        responses=abcd_vars.diagnoses.features,
        random=True,
        random_state=RANDOM_STATE
    )
    cc.fit(train, valid)
    
    preds = {}
    for ds_str, ds in zip(['train', 'valid', 'test'], [train, valid, test]):
        manager.save_predictions(
            dataset_name='unpermuted',
            model_name='logreg_cc',
            fold=i,
            split_set=ds_str,
            y_true=ds[abcd_vars.diagnoses.features],
            y_pred=cc.predict_proba(ds[features_selected])
        )
        
manager.finish()

C:\ProgramData\Anaconda3\envs\abcd\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\envs\abcd\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


### First full experiment (restart kernel before running below code)

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from pytorch_lightning.loggers import TensorBoardLogger

from definitions import REPO_ROOT
import src.data.preprocess_data as prep
from src.data.data_loader import RepeatedStratifiedKFoldDataloader
from src.models.classifier_chain import ClassifierChainEnsemble
from src.models.logistic_regression import (
    LogisticRegressionOVRPredictor, LogisticRegressionModel
)
from src.models.xgboost_pipeline import DepthwiseXGBPipeline
import src.data.var_names as abcd_vars
from src.models.evaluation import (
    MultilabelBinaryEvaluator, BinaryEvaluator, ResultManager
)

RANDOM_STATE = 77

In [2]:
abcd_data = pd.read_csv(REPO_ROOT / 'data' / 'processed' / 'abcd_data.csv', index_col='src_subject_id')
abcd_data = prep.select_one_child_per_family(
    abcd_data_path=REPO_ROOT / 'data' / 'raw',
    abcd_df=abcd_data,
    random_state=RANDOM_STATE
)
abcd_data.shape

(7191, 304)

In [3]:
data_loader = RepeatedStratifiedKFoldDataloader(
    dataframe=abcd_data,
    features=abcd_vars.all_brain_features.features,
    responses=abcd_vars.diagnoses.features,
    confounders=abcd_vars.sociodem.features,
    n=1,
    k=5,
    val_ratio=0.2,
    random_state=RANDOM_STATE
)

tensorboard_logger = TensorBoardLogger(REPO_ROOT / 'tensorboard')
manager = ResultManager(
    tensorboard_logger=tensorboard_logger,
    save_root=REPO_ROOT / 'results',
    save_params={
        'random_state': RANDOM_STATE, 'n': data_loader.n, 'k': data_loader.k
    }
)

In [17]:
logistic_regression_args = {
    'solver': 'lbfgs',
    'max_iter': 500,
    'class_weight': 'balanced'
}

In [ ]:
for i, (train, valid, test, features_selected) in enumerate(data_loader):

    # Logistic regression OVR predictor
    ovr_predictor = LogisticRegressionOVRPredictor(
        features=features_selected,
        responses=abcd_vars.diagnoses.features,
        model_args=logistic_regression_args,
        random_state=RANDOM_STATE
    )
    ovr_predictor.fit(pd.concat((train, valid)))
    preds = {}
    for ds_str, ds in zip(['train', 'valid', 'test'], [train, valid, test]):
        manager.save_predictions(
            dataset_name='unpermuted',
            model_name='logistic_regression_ovr',
            fold=i,
            split_set=ds_str,
            y_true=ds[abcd_vars.diagnoses.features],
            y_pred=ovr_predictor.predict(ds[features_selected])
        )
        
    # Logistic regression classifier chain ensemble
    lr_cce_predictor = ClassifierChainEnsemble(
        model=LogisticRegressionModel,
        features=features_selected,
        responses=abcd_vars.diagnoses.features,
        num_chains=10,
        model_args=logistic_regression_args,
        random_state=RANDOM_STATE
    )
    cce_predictor.fit(train, valid)
    preds = {}
    for ds_str, ds in zip(['train', 'valid', 'test'], [train, valid, test]):
    manager.save_predictions(
        dataset_name='unpermuted',
        model_name='logistic_regression_cce',
        fold=i,
        split_set=ds_str,
        y_true=ds[abcd_vars.diagnoses.features],
        y_pred=cce_predictor.predict(ds[features_selected])
    )
    """
    # XGBoost classifier chain ensemble
    xgboost_cce_predictor = ClassifierChainEnsemble(
        model=DepthwiseXGBPipeline,
        features=features_selected,
        responses=abcd_vars.diagnoses.features,
        num_chains=10,
        model_args={
            'n_calls': 30
        },
        random_state=RANDOM_STATE
    )
    xgboost_cce_predictor.fit(train, valid)
    preds = {}
    for ds_str, ds in zip(['train', 'valid', 'test'], [train, valid, test]):
    manager.save_predictions(
        dataset_name='unpermuted',
        model_name='xgboost_cce',
        fold=i,
        split_set=ds_str,
        y_true=ds[abcd_vars.diagnoses.features],
        y_pred=xgboost_cce_predictor.predict(ds[features_selected])
    )"""
        
manager.finish()